

```
#Pre-requisites
```



In [66]:
# Installing pyspark
#
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [65]:
# Next, we'll run a local spark session
# ---
#
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

Start a spark session and load the stock file while inferring the data types.


In [47]:
df = spark.read.csv("/content/saf_stock (1).csv", inferSchema=True, header=True)


 Determine the column names

In [48]:
columns = df.columns
print(columns)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']


Make observations about the schema.


Ans:The data types of the columns contain intergers,it has seven columns including date , open stock price of the day  ,high stock price of the day,  low stock price of the day, closing stock price of the day, volume that include the number shares traded per day and adjusted closing stock price per day .

Show the first 5 rows


In [49]:
df.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



Use the describe method to learn about the data frame

In [50]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

Data Preparation


In [51]:
from pyspark.sql.functions import format_number


Format all the data to 2 decimal places i.e. format_number()

In [ ]:
df.select(df.columns[1:]).describe().select(
    format_number("Open",2).alias("Open"),
    format_number("High",2).alias("High"),
    format_number("Low",2).alias("Low"),
    format_number("Close",2).alias("Close")
    format_number("summary",2).alias("summary")
    format_number("Volume",2).alias("Volume")
    format_number("Adj Close",2).alias("Adj Close")
).show()

Create a new data frame with a column called HV Ratio that is the ratio of the
High Price versus volume of stock traded for a day

In [55]:
from pyspark.sql.functions import col

In [57]:
df_hv = df.withColumn("HV Ratio", col("High")/col("Volume"))
df_hv

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: int, Adj Close: double, HV Ratio: double]

Data Analysis


What day had the Peak High in Price?


In [61]:
from pyspark.sql.functions import max


In [62]:
df_high = df.select(max("High").alias("High"))
df.select("Date", "High").where(df["High"] == df_high.collect()[0][0]).show()

+-------------------+---------+
|               Date|     High|
+-------------------+---------+
|2015-01-13 00:00:00|90.970001|
+-------------------+---------+



What is the mean of the Close column?


In [68]:
from pyspark.sql.functions import mean
df.select(mean("Close")).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



What is the max and min of the Volume column?


In [70]:
from pyspark.sql.functions import max, min
df.select(max("Volume"), min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



How many days was the Close lower than 60 dollars?

In [71]:
from pyspark.sql.functions import count

df.filter(df["Close"] < 60).select(count("Close")).show()

+------------+
|count(Close)|
+------------+
|          81|
+------------+



What percentage of the time was the High greater than 80 dollars?


In [72]:
from pyspark.sql.functions import count, lit

total = df.select(count("High")).collect()[0][0]
greater_than_80 = df.filter(df["High"] > 80).select(count("High")).collect()[0][0]
percentage = (greater_than_80 / total) * 100
print(str(round(percentage, 2)) + "%")

9.14%


What is the Pearson correlation between High and Volume?


In [73]:
from pyspark.sql.functions import corr

df.select(corr("High", "Volume")).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



What is the max High per year?


In [78]:
from pyspark.sql.functions import year, max

df.groupBy(year("Date").alias("Year")).agg(max("High")).orderBy("Year").show()

+----+---------+
|Year|max(High)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



What is the average Close for each Calendar Month?

In [77]:
from pyspark.sql.functions import month, avg

df.groupBy(month("Date").alias("Month")).agg(avg("Close")).orderBy("Month").show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

